In [1]:
env_name = "/Users/rubelmondol/HDILab/Custom training/envs/RollerBall"  # Name of the Unity environment binary to launch
train_mode = True  # Whether to run the environment in training or inference mode

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sys

from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfig, EngineConfigurationChannel

import torch
import time
import random
import numpy as np
from collections import deque
from dqn_agent import Agent

%matplotlib inline

print("Python version:")
print(sys.version)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
num_episodes=2000
epsilon=1.0
epsilon_min=0.05
epsilon_decay=0.99
scores = []
scores_average_window = 100      
solved_score = 14  

In [4]:
engine_configuration_channel = EngineConfigurationChannel()
# env = UnityEnvironment(base_port = UnityEnvironment.DEFAULT_EDITOR_PORT, file_name=env_name, side_channels = [engine_configuration_channel])
env = UnityEnvironment(base_port = 5004, file_name=env_name, side_channels = [engine_configuration_channel])


#Reset the environment
env.reset()
# env.render()

# Set the default brain to work with
group_name = env.get_agent_groups()[0]
print(group_name)
group_spec = env.get_agent_group_spec(group_name)

# Set the time scale of the engine
engine_configuration_channel.set_configuration_parameters(time_scale = 3.0)

INFO:mlagents_envs:Connected new brain:
RollerBallBrain?team=0


RollerBallBrain?team=0


In [5]:
# Get the state of the agents
step_result = env.get_step_result(group_name)

# Examine the number of observations per Agent
print("Number of observations : ", len(group_spec.observation_shapes))
print(group_spec.observation_shapes[0][0])

state_size = group_spec.observation_shapes[0][0]

action_size = group_spec.action_size
# Examine the state space for the first observation for all agents
print("Agent state looks like: \n{}".format(step_result.obs[0]))

# Examine the state space for the first observation for the first agent
print("Agent state looks like: \n{}".format(step_result.obs[0][0]))

# Is there a visual observation ?
vis_obs = any([len(shape) == 3 for shape in group_spec.observation_shapes])
print("Is there a visual observation ?", vis_obs)

# Examine the visual observations
if vis_obs:
    vis_obs_index = next(i for i,v in enumerate(group_spec.observation_shapes) if len(v) == 3)
    print("Agent visual observation look like:")
    obs = step_result.obs[vis_obs_index]
    plt.imshow(obs[0,:,:,:])

Number of observations :  1
11
Agent state looks like: 
[[ 5.8897753   1.5         5.88577     9.999987    0.9732826   0.58424544
   8.2         3.76       -9.8         0.          0.        ]]
Agent state looks like: 
[ 5.8897753   1.5         5.88577     9.999987    0.9732826   0.58424544
  8.2         3.76       -9.8         0.          0.        ]
Is there a visual observation ? False


In [6]:
print(state_size, action_size)
agent = Agent(state_size=state_size, action_size=action_size, dqn_type='DQN')

11 2


In [7]:
for episode in range(10):
    env.reset()
    step_result = env.get_step_result(group_name)
    done = False
    episode_rewards = 0
    while not done:
        action_size = group_spec.action_size
        print( group_spec.is_action_continuous())
        if group_spec.is_action_continuous():
            action = np.random.randn(step_result.n_agents(), group_spec.action_size)
        
            
        if group_spec.is_action_discrete():
            branch_size = group_spec.discrete_action_branches
            action = np.column_stack([np.random.randint(0, branch_size[i], size=(step_result.n_agents())) for i in range(len(branch_size))])
        print(action)
        env.set_actions(group_name, action)
        env.step()
        step_result = env.get_step_result(group_name)
        print(step_result)
        episode_rewards += step_result.reward[0]
        done = step_result.done[0]
        break
    print("Total reward this episode: {}".format(episode_rewards))
    break

True
[[-0.47611294 -2.5712315 ]]
Total reward this episode: -0.09999998658895493


In [8]:
for i_episode in range(1, num_episodes+1):

    # reset the unity environment at the beginning of each episode
    env_info = env.reset()

    step_result = env.get_step_result(group_name)
    # get initial state of the unity environment 
    state = step_result.obs[0][0]
    
    print(type(state))

    # set the initial episode score to zero.
    score = 0

    # Run the episode training loop;
    # At each loop step take an epsilon-greedy action as a function of the current state observations
    # Based on the resultant environmental state (next_state) and reward received update the Agent network
    # If environment episode is done, exit loop...
    # Otherwise repeat until done == true 
    while True:
        # determine epsilon-greedy action from current sate
#         print((agent.act(state, epsilon).cpu().detach()))
        action = agent.act(state, epsilon)#unsqueeze(0).numpy()
        action = np.expand_dims(action, axis=0)
        print(action)

        # send the action to the environment and receive resultant environment information
        env.set_actions(group_name, action)
        env_info = env.step()
        step_result = env.get_step_result(group_name)

        next_state = step_result.obs[0][0]   # get the next state
        print(len(next_state))
        reward = step_result.reward                   # get the reward
        done = step_result.done                 # see if episode has finished

        #Send (S, A, R, S') info to the DQN agent for a neural network update
        agent.step(state, action, reward, next_state, done)

        # set new state to current state for determining next action
        state = next_state

        # Update episode score
        score += reward

        # If unity indicates that episode is done, 
        # then exit episode loop, to begin new episode
        if done:
            break

    # Add episode score to Scores and...
    # Calculate mean score over last 100 episodes 
    # Mean score is calculated over current episodes until i_episode > 100
    scores.append(score)
    average_score = np.mean(scores[i_episode-min(i_episode,scores_average_window):i_episode+1])

    # Decrease epsilon for epsilon-greedy policy by decay rate
    # Use max method to make sure epsilon doesn't decrease below epsilon_min
    epsilon = max(epsilon_min, epsilon_decay*epsilon)

    # (Over-) Print current average score
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score), end="")

    # Print average score every scores_average_window episodes
    if i_episode % scores_average_window == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score))
    
    # Check to see if the task is solved (i.e,. avearge_score > solved_score). 
    # If yes, save the network weights and scores and end training.
    if average_score >= solved_score:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, average_score))

        # Save trained neural network weights
        timestr = time.strftime("%Y%m%d-%H%M%S")
        nn_filename = "dqnAgent_Trained_Model_" + timestr + ".pth"
        torch.save(agent.network.state_dict(), nn_filename)

        # Save the recorded Scores data
        scores_filename = "dqnAgent_scores_" + timestr + ".csv"
        np.savetxt(scores_filename, scores, delimiter=",")
        break

<class 'numpy.ndarray'>
tensor([[-0.3235, -0.9126]])
[[-0.32351932 -0.9126088 ]]
11
State:
[-13.647765    1.5        10.491365    9.696502    1.4999998  -1.054703
   8.2         3.76       -9.8        -1.454345   -7.854128 ]
tensor([[-0.2776, -0.8995]])
[[-0.277585   -0.89948153]]
11
State:
[-13.647765    1.5        10.491365    9.366691    1.5000005  -2.702525
   8.2         3.76       -9.8        -1.8153007  -8.612639 ]
tensor([[-0.2355, -0.8708]])
[[-0.23548731 -0.87080693]]
11
State:
[-13.647765    1.5        10.491365    8.960596    1.5000011  -4.5636425
   8.2         3.76       -9.8        -2.20616    -9.871385 ]
tensor([[ 0.2085, -1.0910]])
[[ 0.2085022 -1.0910466]]
11
State:
[-13.647765     1.5         10.491365     8.526749     1.5367018
  -6.343897     8.2          3.76        -9.8         -0.9453567
   0.26572132]
tensor([[ 0.2064, -1.0937]])
[[ 0.20642474 -1.0937235 ]]
11
State:
[-13.647765     1.5         10.491365     8.379046     1.8189843
  -6.304557     8.2          3

11
State:
[-1.3647765e+01  1.5000000e+00  1.0491365e+01  4.2375626e+01
  1.5000124e+00 -4.6336128e+01  8.1999998e+00  3.7600000e+00
 -9.8000002e+00  7.6942391e+00  3.9224774e-02]
tensor([[ 1.0610, -2.2894]])
[[ 1.0609974 -2.2894354]]
11
State:
[-1.3647765e+01  1.5000000e+00  1.0491365e+01  4.3913994e+01
  1.5000113e+00 -4.6328274e+01  8.1999998e+00  3.7600000e+00
 -9.8000002e+00  7.6920619e+00  3.9280832e-02]
tensor([[ 1.0363, -2.3137]])
[[ 1.0363171 -2.3136694]]
11
State:
[-1.3647765e+01  1.5000000e+00  1.0491365e+01  4.5451263e+01
  1.5000112e+00 -4.6320423e+01  8.1999998e+00  3.7600000e+00
 -9.8000002e+00  7.6860971e+00  3.9250404e-02]
tensor([[ 1.0118, -2.3385]])
[[ 1.0118136 -2.338479 ]]
11
State:
[-1.3647765e+01  1.5000000e+00  1.0491365e+01  4.6987305e+01
  1.5000120e+00 -4.6312580e+01  8.1999998e+00  3.7600000e+00
 -9.8000002e+00  7.6799150e+00  3.9181143e-02]
tensor([[ 0.9505, -2.0704]])
[[ 0.95053273 -2.0704324 ]]
11
State:
[-1.3647765e+01  1.5000000e+00  1.0491365e+01  4.747

RuntimeError: Invalid index in gather at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:654

In [9]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.
